In [3]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client.sample_mflix
movies = db.movies

print(movies)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'sample_mflix'), 'movies')


#### find() => aggrigation Framework문법을 사용하면 pipeline 생성해서 복잡한 데이터 처릴 순차적으로
1) \$match => 특정 조건에 해당되는 문서를 찾아오고자 할 때
2) \$limit => find().limit(2)와 동일한 기능
3) \$group => SQL에서 GROUP BY와 동일한 기능
4) \$sort => 문서의 정렬을 하고 싶을 때 복수의 정렬을 할 수 있음 다만 좌측부터 정렬방식이 적용
5) \$project => 특정 원하는 값만 선택해서 출력하고자 할 때 사용
6) \$unwind => 배열 안에 입력된 값을 추출해서 출력하고자 할 때

In [7]:
for movie in movies.aggregate([{"$match" : {"genres" : "Action"}},{"$limit" : 1}]) :
    print(movie)

{'_id': ObjectId('573a1390f29313caabcd5293'), 'plot': "Young Pauline is left a lot of money when her wealthy uncle dies. However, her uncle's secretary has been named as her guardian until she marries, at which time she will officially take ...", 'genres': ['Action'], 'runtime': 199, 'cast': ['Pearl White', 'Crane Wilbur', 'Paul Panzer', 'Edward Josè'], 'num_mflix_comments': 1, 'poster': 'https://m.media-amazon.com/images/M/MV5BMzgxODk1Mzk2Ml5BMl5BanBnXkFtZTgwMDg0NzkwMjE@._V1_SY1000_SX677_AL_.jpg', 'title': 'The Perils of Pauline', 'fullplot': 'Young Pauline is left a lot of money when her wealthy uncle dies. However, her uncle\'s secretary has been named as her guardian until she marries, at which time she will officially take possession of her inheritance. Meanwhile, her "guardian" and his confederates constantly come up with schemes to get rid of Pauline so that he can get his hands on the money himself.', 'languages': ['English'], 'released': datetime.datetime(1914, 3, 23, 0, 0), '

In [23]:
pipeline = [{"$group" : {"_id" : "$directors", "count" : {"$sum" : 1}}}, {"$limit" : 5}]

for movie in movies.aggregate(pipeline) :
    print(movie)

{'_id': ['Catherine Owens', 'Mark Pellington'], 'count': 1}
{'_id': ['Lance W. Dreesen'], 'count': 1}
{'_id': ['Iain Softley'], 'count': 5}
{'_id': ['Clay Liford'], 'count': 1}
{'_id': ['Konstantin Maksimov'], 'count': 1}


In [15]:
pipeline = [{"$sort" : {"title" : 1}}, {"$limit" : 3}]

for movie in movies.aggregate(pipeline) :
    print(movie)

{'_id': ObjectId('68b94fb652698935b8441531'), 'users': '홍길동', 'email': 'hong@test.com', 'genres': ['Action', 'Comedy'], 'comments': 'Action 영화 진짜 재밌다!'}
{'_id': ObjectId('573a13def29313caabdb6863'), 'plot': "Set in the golden era of Grand Prix Racing '1' tells the story of a generation of charismatic drivers who raced on the edge, risking their lives during Formula 1's deadliest period, and the men who stood up and changed the sport forever.", 'genres': ['Documentary'], 'runtime': 112, 'cast': ['Michael Fassbender', 'Niki Lauda', 'Lewis Hamilton', 'Michael Schumacher'], 'num_mflix_comments': 1, 'poster': 'https://m.media-amazon.com/images/M/MV5BMjAxNDI5NTMxMF5BMl5BanBnXkFtZTgwMDMzMjg4MTE@._V1_SY1000_SX677_AL_.jpg', 'title': 1, 'fullplot': "Set in the golden era of Grand Prix Racing '1' tells the story of a generation of charismatic drivers who raced on the edge, risking their lives during Formula 1's deadliest period, and the men who stood up and changed the sport forever.", 'languages

In [16]:
pipeline = [{"$project" : {"_id" : 0, "title" : 1, "genres" : 1,}}, {"$limit" : 3}]

for movie in movies.aggregate(pipeline) :
    print(movie)

{'genres': ['Short'], 'title': 'Blacksmith Scene'}
{'genres': ['Short', 'Western'], 'title': 'The Great Train Robbery'}
{'genres': ['Short', 'Drama', 'Fantasy'], 'title': 'The Land Beyond the Sunset'}


In [17]:
pipeline = [{"$unwind" : "$genres"}, {"$limit" : 3}]

for movie in movies.aggregate(pipeline) :
    print(movie)

{'_id': ObjectId('573a1390f29313caabcd4135'), 'plot': 'Three men hammer on an anvil and pass a bottle of beer around.', 'genres': 'Short', 'runtime': 1, 'cast': ['Charles Kayser', 'John Ott'], 'num_mflix_comments': 1, 'title': 'Blacksmith Scene', 'fullplot': 'A stationary camera looks at a large anvil with a blacksmith behind it and one on either side. The smith in the middle draws a heated metal rod from the fire, places it on the anvil, and all three begin a rhythmic hammering. After several blows, the metal goes back in the fire. One smith pulls out a bottle of beer, and they each take a swig. Then, out comes the glowing metal and the hammering resumes.', 'countries': ['USA'], 'released': datetime.datetime(1893, 5, 9, 0, 0), 'directors': ['William K.L. Dickson'], 'rated': 'UNRATED', 'awards': {'wins': 1, 'nominations': 0, 'text': '1 win.'}, 'lastupdated': '2015-08-26 00:03:50.133000000', 'year': 1893, 'imdb': {'rating': 6.2, 'votes': 1189, 'id': 5}, 'type': 'movie', 'tomatoes': {'vi

In [22]:
# $group & 집계연산자 응용 패턴

pipeline = [
    {"$group" : {"_id" : "$directors", "count" : {"$sum" : 1}}}, {"$limit" : 5}
]

for movie in movies.aggregate(pipeline) :
    print(movie)

{'_id': ['Catherine Owens', 'Mark Pellington'], 'count': 1}
{'_id': ['Lance W. Dreesen'], 'count': 1}
{'_id': ['Iain Softley'], 'count': 5}
{'_id': ['Clay Liford'], 'count': 1}
{'_id': ['Konstantin Maksimov'], 'count': 1}


In [21]:
pipeline = [
    {"$group" : {"_id" : "$directors", "average_rating" : {"$avg" : "$imdb.rating"}}},
    {"$limit" : 5}
]

for movie in movies.aggregate(pipeline) :
    print(movie)

{'_id': ['Catherine Owens', 'Mark Pellington'], 'average_rating': 8.4}
{'_id': ['Lance W. Dreesen'], 'average_rating': 5.4}
{'_id': ['Iain Softley'], 'average_rating': 6.779999999999999}
{'_id': ['Clay Liford'], 'average_rating': 4.2}
{'_id': ['Konstantin Maksimov'], 'average_rating': 3.8}


In [24]:
# 컬렉션에 있는 영화의 갯수를 계산하세요
# 특정 컬렉션에 존재하는 갯수를 찾고자 할 때 : count_documents()

movie_count = movies.count_documents({})
print("영화 수 : ",  movie_count)

영화 수 :  23540


In [31]:
# 평균 영화 길이를 찾으세요
# 영화 필드를 그룹화하고 해당 영화들의 런타임 값을 활용해서 평균값을 구한다

# for movie in movies.aggregate([{"$group" : {"_id" : None, "avg_length" : {"$avg" : "$runtime"}}}]):
#    print(movie)

result = list(movies.aggregate([{"$group" : {"_id" : None, "avg_length" : {"$avg" : "$runtime"}}}]))
result[0]["avg_length"]

103.78932097696172

In [40]:
# 각 장르에 대한 영화 갯수를 계산하세요
# 장르를 기준으로 갯수 출력 => $group // $sum

genre_counts = list(movies.aggregate(
    [
        {"$unwind" : "$genres"},
        {"$group" : {"_id" : "$genres", "count" : {"$sum" : 1}}}
    ]
))

for genre in genre_counts :
    print(f"장르 : {genre["_id"]}, 영화 수 : {genre["count"]}개")

장르 : Western, 영화 수 : 274개
장르 : History, 영화 수 : 999개
장르 : Music, 영화 수 : 840개
장르 : Short, 영화 수 : 478개
장르 : Comedy, 영화 수 : 7025개
장르 : Mystery, 영화 수 : 1259개
장르 : Crime, 영화 수 : 2678개
장르 : Action, 영화 수 : 2540개
장르 : Family, 영화 수 : 1311개
장르 : Thriller, 영화 수 : 2658개
장르 : Sport, 영화 수 : 390개
장르 : Adventure, 영화 수 : 2045개
장르 : Musical, 영화 수 : 487개
장르 : War, 영화 수 : 794개
장르 : Horror, 영화 수 : 1703개
장르 : Sci-Fi, 영화 수 : 1034개
장르 : Documentary, 영화 수 : 2129개
장르 : Film-Noir, 영화 수 : 105개
장르 : Talk-Show, 영화 수 : 1개
장르 : Biography, 영화 수 : 1404개
장르 : Drama, 영화 수 : 13789개
장르 : Romance, 영화 수 : 3665개
장르 : Animation, 영화 수 : 971개
장르 : Fantasy, 영화 수 : 1153개
장르 : News, 영화 수 : 51개


In [46]:
# 2015년 이후에 개봉한 영화를 제목으로 정렬하여 나열해주세요

# find()
post_2015 = movies.find({"year" : {"$gt" : 2015}}).sort("title")

for movie in post_2015 :
    print(f"영화제목 : {movie["title"]}")

영화제목 : The Masked Saint


In [49]:
# aggrigate()
pipeline = [
    {"$match" : {"year" : {"$gt" : 2015}}},
    {"$sort" : {"title" : 1}},
    {"$project" : {"_id" : 0, "title" : 1}}
]
for movie in movies.aggregate(pipeline) : 
    print(f"영화제목 : {movie["title"]}")

영화제목 : The Masked Saint


In [53]:
# 가장 많은 영화를 제작하는 상위 5개 국가를 찾아주세요
pipeline = [
    {"$unwind" : "$countries"},
    {"$group" : {"_id" : "$countries", "count" : {"$sum" : 1}}},
    {"$sort" : {"count" : -1}},
    {"$limit" : 5}
]

top_countries = movies.aggregate(pipeline)
for country in top_countries : 
    print(f"국가 : {country["_id"]}, 영화수 : {country["count"]}개")

국가 : USA, 영화수 : 11855개
국가 : France, 영화수 : 3093개
국가 : UK, 영화수 : 2904개
국가 : Germany, 영화수 : 1659개
국가 : Italy, 영화수 : 1388개
